# Einsum Puzzles
This Colab will teach you how to use numpy.einsum through examples.

In [ ]:
# einsum is available nunmpy, torch and JAX, so the following would work:
# np.einsum, torch.einsum, jnp.einsum
# Let's start with numpy

import numpy as np

def check_answer(expected, given):
  if expected.shape != given.shape:
    print("Shape mismatch!")
    print(f"Expected shape: {expected.shape}")
    print(f"Given shape: {given.shape}")
    return
  if np.all(expected == given):
    print("Correct!")
  else:
    print("Incorrect!")

Einsum allows you to create operations in which multiple inputs (vectors, arrays or, more generally, tensors of any rank) are combined to create a single result. A typical call would look like this:
```
d = np.einsum("◻◻,◻◻◻,◻◻->◻◻", a, b, c)
```
where `a`, `b`, and `c` are inputs and `d` is the results. The subscripts string `"◻◻,◻◻◻,◻◻->◻◻"` has two parts separated by `->`:
- The left part describes the dimensions of the inputs. There will be one description per input and they are separated by commas.
- The right part describes the dimensions of the result.

Here's our first example:

In [ ]:
vector_length = 5
u = np.arange(vector_length)
print(f"{u.shape=}")
print(f"u: {u}")

r = np.einsum("i->i", u)
print(f"{r.shape=}")
print(f"r: {r}")

In the example above, there was one input `u` which was a vector. The left part of the subscripts string was `"i"`. The use of a single letter means that the input has one dimension. The output is also specified as `"i"`, so it ends up being identical to the input.

The choice of the letter does't matter, so instead of `"i"` we could have used `"x"` or any other letter with the same result:

In [ ]:
r = np.einsum("x->x", u)
print(f"r: {r}")

There is a shortcut for this operation. If we completely omit the `->` symbol and the right hand side, we can get as output the same vector as the input.

In [ ]:
r = np.einsum("i", u)
print(f"r: {r}")

Another important feature of einsum is that if the same letter appears twice on the left hand side then corresponding elements from the two dimensions described by that letter will be pairwise multiplied by each other.

In the example below, the result, `r`, is a vector of the same length as the two inputs `u` and `v` and each of its elements is a product of two corresponding elements of `u` and `v`, so we want to compute `r` such that:

$$
r_i = u_i v_i
$$

In [ ]:
print(f"u: {u}")
v = np.arange(vector_length, 2 * vector_length)
print(f"v: {v}")
print(f"{v.shape=}")

r = np.einsum("i,i->i", u, v)
print(f"r: {r}")

# Puzzle 1: inner product

Use your knowledge from the previous two examples to compute the inner product of `u` and `v`, that is compute a value which is the sum of pairwise products of elements of `u` and `v`. Remember that:
- Repeating the same index twice on the left hand side of `->` means that elements will be pairwise multiplied.
- Omitting an index on the right hand side of `->` means that we will sum all values along that dimension.

Inner product is also implemented as `np.inner` and we will use that function to check the correctness of the answer.

The result value should be:

$$
\sum_i u_i v_i
$$

In [ ]:
print(f"u: {u}")
print(f"v: {v}")

# Fill in the first argument to the np.einsum call below:
inner_product = np.einsum("", u, v)
print(f"r: {inner_product}")

check_answer(np.inner(u, v), inner_product)

The same principle applies to matrices. Consider this snippet which doesn't change the input.

In [ ]:
rows, cols = 3, 4
a = np.arange(rows * cols).reshape(rows, cols)
print(f"{a.shape=}")
print(f"a:\n{a}")

r = np.einsum("ij->ij", a)
print(f"{r.shape=}")
print(f"r:\n{r}")

# Puzzle 2: matrix transpose

Since the order of letters in the subscripts string determines the order of dimensions, you can take advantage of that to permute dimensions of a tensor. How would you use that to transpose an array? Fill in the subsripts string below to solve this puzzle.

In [ ]:
# Fill in the first argument to the np.einsum call below:
a_transpose = np.einsum("", a)
print(f"{a_transpose.shape=}")
print(f"a^T:\n{a_transpose}")

check_answer(a.T, a_transpose)

To summarize what we learnt so far:

- Letters which appear in the subscripts string both to the left and right of the `"->"` arrow are called *free indices*. They indicate no change the dimension denoted by that index.

- If an index appears to the left of `"->"` but not to the right is called a *summation index* and `einsum` will add all values along that dimension. We can use this to calculate the sum of all elements of our vector `u`.

- If an index appears twice on the left hand side, we will perform pairwise multiplication of the corresponding elements.

Consider another summation example. This time we have an array with 3 rows and 4 columns and we want to add all elements in a given row, so we want to obtain a vector of three elements in which the first element is the sum of all 4 elements in the first row of the array and so on:

$$
r_i = \sum_j a_{i,j}
$$

We make this happen by not listing the column dimension (denoted by `j`) on the right hand side:

In [ ]:
rows, cols = 3, 4
a = np.arange(rows * cols).reshape(rows, cols)
print(f"{a.shape=}")
print(f"a:\n{a}")

r = np.einsum("ij->i", a)
print(f"{r.shape=}")
print(f"r:\n{r}")

# Puzzle 3: Summing columns of an array

How would you create a vector which contains a value for the sum of each columns of the array? For our array of 3 rows and 4 columns, it should be a vector of 4 elements, each of them being the sum of the three corresponding elements.

$$
r_j = \sum_i a_{i,j}
$$

In [ ]:
# Fill in the first argument to the np.einsum call below:
spec = np.sum(a, axis=0)

print(f"spec: {spec}")
r = np.einsum("ij->j", a)
print(f"{r.shape=}")
print(f"r:\n{r}")

check_answer(spec, r)

# A note on vectors in this colab

Some of the examples operate on vectors and arrays. Depending on the type of operation, the vector will be either a row vector or a column vector. However when Python prints a numpy vector, it is always printed in a single line of text which makes is appear like a row vector. If you want to visualize if the vector is a row or column, you will need to use your knowledge of linear algebra.

The two most common operations we will encounter are:
- multiplication of a vector by matrix: the vector is a *row vector* of length equal to the number of rows of the matrix. The result is a *row vector*.
- multiplication of a matrix by a vector: the vector is a *column vector* of length equal to the number of columns of the matrix. The result is a *column vector*.

In [ ]:
# Create a matrix and two vectors of the matching lengths

rows, cols = 3, 4
a = np.arange(rows * cols).reshape(rows, cols)
print(f"{a.shape=}")
print(f"a:\n{a}")

u = np.arange(cols)
print(f"{u.shape=}")
print(f"u: {u}")

w = np.arange(rows)
print(f"{w.shape=}")
print(f"w: {w}")

print("\nVector multiplied by matrix (the result is a row vector)")
wxa = w @ a
print(f"{wxa.shape=}")
print(f"wxa: {wxa}")


print("\nMatrix multiplied by vector (the result is a column vector)")
axu = a @ u
print(f"{axu.shape=}")
print(f"axu: {axu}")

print("\nUse matrix transpose to compute wxa as a column vector.")
wxa2 = a.T @ w
print(f"{wxa2.shape=}")
print(f"wxa2:\n{wxa2}")
print("In math vectors wxa and wxa2 would have be row and column.")
print("But in Python they're printed the same way and have the same shape.")

print("\nThese two vectors are equal to each other.")
print(f"{np.array_equal(wxa, wxa2)=}")

# Puzzle 4: Multiply a matrix by a vector

How would you multiply a matrix by a vector? The vector is of the length equal to the number of columns in the matrix and the result is a vector of length equal to the number of rows of the matrix.

So, the i'th element of the result can be expressed as:
$$
r_i = \sum_j a_{i,j}u_j
$$

In [ ]:
print(f"{a.shape=}")
print(f"a:\n{a}")

print(f"{u.shape=}")
print(f"u: {u}")

spec = a @ u
print(f"{spec.shape=}")
print(f"spec: {spec}")

r = np.einsum('', a, u)
print(f"{r.shape=}")
print(f"r:\n{r}")

check_answer(spec, r)

# Puzzle 5: Matrix multiplication

Given what we've learned so far about `einsum`, matrix multiplication is easy to express since we perform an inner product along two matching dimensions and we already did this in Puzzle 1. The other two dimensions are unchanged.

In [ ]:
rows2 = cols
cols2 = 5
b = np.arange(rows2 * cols2).reshape(rows2, cols2)
spec = a @ b

r = np.einsum('', a, b)
check_answer(spec, r)